In [3]:
import os 
import re
import random
import numpy as np 
import pandas as pd
from tqdm import tqdm,tqdm_notebook
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

import multiprocessing
from multiprocessing import Pool

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

import lightgbm as lgb

import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
train_err = pd.read_csv('dataset/train_err_data.csv')
train_quality = pd.read_csv('dataset/train_quality_data.csv')

In [3]:
test_err = pd.read_csv('dataset/test_err_data.csv')
test_quality = pd.read_csv('dataset/test_quality_data.csv')

In [4]:
train_quality = train_quality.drop(['quality_3', 'quality_4'], axis=1)
test_quality = test_quality.drop(['quality_3', 'quality_4'], axis=1)

In [5]:
fwver_model_dict = {'04.22': 'model_0',
                    '04.16': 'model_1',
                    '04.33': 'model_2',
                    '05.15': 'model_3',
                    '03.11': 'model_4',
                    '04.82': 'model_5',
                    '05.66': 'model_7',
                    '04.73': 'model_8'}

def case_fwver(x):
    try:    
        return fwver_model_dict[x[:5]]
    except:
        return None

In [6]:
exist_case = train_quality[['user_id', 'time', 'fwver']].drop_duplicates().reset_index(drop=True)
exist_case['model_nm'] = exist_case['fwver'].apply(lambda x: case_fwver(x))


test_exist_case = test_quality[['user_id', 'time', 'fwver']].drop_duplicates().reset_index(drop=True)
test_exist_case['model_nm'] = test_exist_case['fwver'].apply(lambda x: case_fwver(x))

In [7]:
train_all = pd.concat([train_err, exist_case], axis=0)
train_all = train_all.groupby('user_id').apply(pd.DataFrame.sort_values, 'time').reset_index(drop=True)


test_all = pd.concat([test_err, test_exist_case], axis=0)
test_all = test_all.groupby('user_id').apply(pd.DataFrame.sort_values, 'time').reset_index(drop=True)

In [8]:
idx = list(train_all.loc[train_all['errtype'].isna()==True].index)
test_idx = list(test_all.loc[test_all['errtype'].isna()==True].index)

In [9]:
train_all['fwver'] = train_all.groupby(['user_id','model_nm'])['fwver'].bfill().ffill()
train_all['model_nm'] = train_all.groupby(['user_id','fwver'])['model_nm'].bfill().ffill()
train_all['errtype'] = train_all.groupby(['user_id','model_nm'])['errtype'].bfill().ffill()
train_all['errcode'] = train_all.groupby(['user_id', 'model_nm'])['errcode'].bfill().ffill()

In [10]:
test_all['fwver'] = test_all.groupby(['user_id','model_nm'])['fwver'].bfill().ffill()
test_all['model_nm'] = test_all.groupby(['user_id','fwver'])['model_nm'].bfill().ffill()
test_all['errtype'] = test_all.groupby(['user_id','model_nm'])['errtype'].bfill().ffill()
test_all['errcode'] = test_all.groupby(['user_id', 'model_nm'])['errcode'].bfill().ffill()

In [15]:
fill_case = train_all.iloc[idx].reset_index(drop=True)
test_fill_case = test_all.iloc[test_idx].reset_index(drop=True)

In [18]:
fill_case.to_csv('./dataset/exist_case.csv', index=False)
test_fill_case.to_csv('./dataset/test_exist_case.csv', index=False)

In [19]:
quality_col = [col for col in train_quality.columns if col.startswith('quality')]

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r',', '', str(x))
    if x =='':
        return 0
    else:
        return float(x)
    
for col in quality_col:
    train_quality[col] = train_quality[col].apply(lambda x: string2num(x))
    test_quality[col] = test_quality[col].apply(lambda x: string2num(x))

In [20]:
fill_case['time'] = fill_case['time'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d%H%M%S')) 
test_fill_case['time'] = test_fill_case['time'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d%H%M%S')) 

In [22]:
train_quality['time'] = train_quality['time'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d%H%M%S')) 
test_quality['time'] = test_quality['time'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d%H%M%S')) 

In [24]:
train_quality = train_quality.drop(['fwver'], axis=1)
train_quality = pd.merge(train_quality, fill_case, how='left', on=['user_id', 'time'])

test_quality = test_quality.drop(['fwver'], axis=1)
test_quality = pd.merge(test_quality, test_fill_case, how='left', on=['user_id', 'time'])

In [38]:
train_quality_missing_col = ['quality_0', 'quality_2', 'quality_5']
test_quality_missing_col = ['quality_0', 'quality_1', 'quality_2', 'quality_5']

In [39]:
for col in train_quality_missing_col:
    train_quality[col] = train_quality.groupby(['user_id', 'model_nm', 'fwver'])[col].ffill().bfill()

In [40]:
for col in test_quality_missing_col:
    test_quality[col] = test_quality.groupby(['user_id', 'model_nm', 'fwver'])[col].ffill().bfill()

In [44]:
train_quality.to_csv('./dataset/fe_train_quality.csv', index=False)
test_quality.to_csv('./dataset/fe_test_quality.csv', index=False)

In [4]:
train_quality = pd.read_csv('./dataset/fe_train_quality.csv', parse_dates=['time'], infer_datetime_format=True)
test_quality =  pd.read_csv('./dataset/fe_test_quality.csv', parse_dates=['time'], infer_datetime_format=True)

In [5]:
def describe_by_time(df):
    # Day,Min With Logs Interval
    log_case = df.groupby(['user_id','time']).size().reset_index(name='cnt')
    log_case['shift_time'] = log_case.groupby(['user_id'])['time'].shift(1)
    log_case['diff_days'] = log_case.apply(lambda x: (x['time'] - x['shift_time']).days, axis=1)
    log_case['diff_mins'] = log_case.apply(lambda x: (x['time'] - x['shift_time']).total_seconds(), axis=1)
    log_case['diff_mins'] /= 60
    
    action_log_interval = log_case.groupby('user_id')['diff_days', 'diff_mins'].agg(['mean', 'std']).reset_index()
    action_log_interval.columns = ['user_id']+['Q_{}_{}'.format(i, j) for i, j in action_log_interval.columns[1:]]
    action_log_interval = action_log_interval.fillna(0)
    
    # Per User Log, time Frequency
    log_cnt = df.groupby('user_id')['time'].nunique().reset_index(name='Q_log_cnt')
    time_cnt = df.groupby(['user_id']).size().reset_index(name='Q_time_cnt')
    
    # Merge
    processed_df = pd.merge(action_log_interval, log_cnt, how='left', on='user_id')
    processed_df = pd.merge(processed_df, time_cnt, how='left', on='user_id')
    
    return processed_df

In [6]:
Q_time = describe_by_time(train_quality)
Q_test_time = describe_by_time(test_quality)

### Side INFO

In [7]:
def make_polycol(df):
    df['errtype'] = df['errtype'].astype('str') 
    df['errcase'] = df.iloc[:]['errtype'] + '_' + df.iloc[:]['errcode']
    df['model_errtype'] = df.iloc[:]['model_nm'] + '_' + df.iloc[:]['errtype']
    df['fwver_errcode'] = df.iloc[:]['fwver'] + '_' + df.iloc[:]['errcode']
    return df

In [8]:
train_quality = make_polycol(train_quality)
test_quality = make_polycol(test_quality)

In [11]:
errcase_dict = {code:idx for idx, code in enumerate(list(train_quality['errcase'].value_counts()[:50].index))}
# model_errtype_dict = {code:idx for idx, code in enumerate(list(train_quality['model_errtype'].value_counts()[:50].index))}
# fwver_errcode_dict = {code:idx for idx, code in enumerate(list(train_quality['fwver_errcode'].value_counts()[:50].index))}

def case_study(dict_case, x):
    try:
        return dict_case[x]
    except:
        return len(dict_case)

def make2group(df):
    df['errcase'] = df['errcase'].apply(lambda x: case_study(errcase_dict, x))
#     df['model_errtype'] = df['model_errtype'].apply(lambda x: case_study(model_errtype_dict, x))
#     df['fwver_errcode'] = df['fwver_errcode'].apply(lambda x: case_study(fwver_errcode_dict, x))
    return df

In [12]:
train_quality = make2group(train_quality)
test_quality = make2group(test_quality)

In [13]:
def side_information(df):
    # Errtype
    errtype_unum = df.groupby(['user_id'])['errtype'].nunique().reset_index(name='Q_errtype_unum')

    # Errcode
    errcode_unum = df.groupby(['user_id'])['errcode'].nunique().reset_index(name='Q_errcode_unum')
    

    # Model_nm
    model_unum = df.groupby(['user_id'])['model_nm'].nunique().reset_index(name='Q_model_unum')
    
#     # Model + Errtype
#     merrtype_unum = df.groupby(['user_id'])['model_errtype'].nunique().reset_index(name='Q_merrtype_unum')


    # Merge
    processed_df = pd.merge(errtype_unum, errcode_unum, how='left', on='user_id')
    
    processed_df = pd.merge(processed_df, model_unum, how='left', on='user_id')
#     processed_df = pd.merge(processed_df, merrtype_unum, how='left', on='user_id')
    return processed_df

In [14]:
Q_side = side_information(train_quality)
Q_test_side = side_information(test_quality)

In [15]:
quality_col = [col for col in train_quality.columns if col.startswith('quality')]

In [27]:
def quality_information(df):
    cumsum_df = df.groupby(['user_id', 'time'])[quality_col].sum().groupby(level=0).cumsum().reset_index()
    cumsum_df = cumsum_df.groupby(['user_id'])[quality_col].agg(['mean', 'std']).reset_index()
    cumsum_df.columns = ['user_id']+['Qcumsum_{}_{}'.format(i, j) for i, j in cumsum_df.columns[1:]]
    
    sum_df = df.groupby(['user_id', 'time'])[quality_col].sum().reset_index()
    sum_df = sum_df.groupby(['user_id'])[quality_col].agg(['mean', 'std']).reset_index()
    sum_df.columns = ['user_id']+['Qsum_{}_{}'.format(i, j) for i, j in sum_df.columns[1:]]
    
    quality_info = df.groupby(['user_id'])[quality_col].agg(['max', 'min', 'mean', 'std']).reset_index()
    quality_info.columns = ['user_id']+['Qavg_{}_{}'.format(i, j) for i, j in quality_info.columns[1:]]
    
    rolling_df = df.groupby(['user_id'])[quality_col].rolling(12).agg(['mean', 'std']).reset_index()
    rolling_df.columns = ['user_id']+['rolling_{}_{}'.format(i, j) for i, j in rolling_df.columns[1:]]
    rolling_df = rolling_df.groupby(['user_id']).mean()
    
    processed_df = pd.merge(cumsum_df, sum_df, how='left', on='user_id')
    processed_df = pd.merge(processed_df, quality_info, how='left', on='user_id')
    processed_df = pd.merge(processed_df, rolling_df, how='left', on='user_id')
#     processed_df = pd.merge(processed_df, sin_df, how='left', on='user_id')
    return processed_df

In [28]:
Q_quality = quality_information(train_quality)
Q_test_quality = quality_information(test_quality)

In [30]:
Q_total = pd.merge(Q_time, Q_side, how='left', on='user_id')
Q_total = pd.merge(Q_total, Q_quality, how='left', on='user_id')

Q_test_total = pd.merge(Q_test_time, Q_test_side, how='left', on='user_id')
Q_test_total = pd.merge(Q_test_total, Q_test_quality, how='left', on='user_id')

In [31]:
Q_total.to_csv('./dataset/train_quality_toal.csv', index=False)
Q_test_total.to_csv('./dataset/test_quality_toal.csv', index=False)

In [29]:
Q_quality.head()

,user_id,Qcumsum_quality_0_mean,Qcumsum_quality_0_std,Qcumsum_quality_1_mean,Qcumsum_quality_1_std,Qcumsum_quality_2_mean,Qcumsum_quality_2_std,Qcumsum_quality_5_mean,Qcumsum_quality_5_std,Qcumsum_quality_6_mean,...,rolling_quality_8_mean,rolling_quality_8_std,rolling_quality_9_mean,rolling_quality_9_std,rolling_quality_10_mean,rolling_quality_10_std,rolling_quality_11_mean,rolling_quality_11_std,rolling_quality_12_mean,rolling_quality_12_std
0,10000,0.0,0.000000,0.0,0.000000,0.0,0.000000,8.00,5.656854,0.0,...,0.0,0.0,0.000000,0.000000e+00,6.000000,1.476063e+00,0.000000,0.000000e+00,0.0,0.0
1,10002,1.0,1.069045,-1.0,1.069045,0.0,1.069045,12.25,9.996428,17.0,...,0.0,0.0,0.076471,5.643772e-02,3.988235,7.901280e-01,-0.023529,5.716582e-02,0.0,0.0
2,10004,-2.0,0.000000,-2.0,0.000000,-2.0,0.000000,1.50,0.707107,41.5,...,0.0,0.0,0.000000,1.490116e-08,2.000000,7.380316e-01,-0.147436,3.515707e-01,0.0,0.0
3,10005,-5.0,7.071068,-5.0,7.071068,-5.0,7.071068,3.00,4.242641,13.0,...,0.0,0.0,0.000000,1.490116e-08,5.000000,7.380316e-01,-0.352564,3.468100e-01,0.0,0.0
4,10006,0.0,0.000000,0.0,0.000000,0.0,0.000000,8.00,4.000000,4.0,...,0.0,0.0,0.000000,1.490116e-08,4.000000,1.853547e-07,0.000000,1.879502e-08,0.0,0.0
